Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# 04. Train in a remote VM (MLC managed DSVM)
* Create Workspace
* Create Project
* Create `train.py` file
* Create DSVM as Machine Learning Compute (MLC) resource
* Configure & execute a run in a conda environment in the default miniconda Docker container on DSVM

## Prerequisites
Make sure you go through the [00. Installation and Configuration](00.configuration.ipynb) Notebook first if you haven't.

In [ ]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

## Initialize Workspace

Initialize a workspace object from persisted configuration.

In [ ]:
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Create Experiment

**Experiment** is a logical container in an Azure ML Workspace. It hosts run records which can include run metrics and output artifacts from your experiments.

In [ ]:
experiment_name = 'train-on-remote-vm'

from azureml.core import Experiment

exp = Experiment(workspace = ws, name = experiment_name)

## View `train.py`

For convenience, we created a training script for you. It is printed below as a text, but you can also run `%pfile ./train.py` in a cell to show the file.

In [ ]:
with open('./train.py', 'r') as training_script:
    print(training_script.read())

## Create Linux DSVM as a compute target

**Note**: If creation fails with a message about Marketplace purchase eligibilty, go to portal.azure.com, start creating DSVM there, and select "Want to create programmatically" to enable programmatic creation. Once you've enabled it, you can exit without actually creating VM.
 
**Note**: By default SSH runs on port 22 and you don't need to specify it. But if for security reasons you switch to a different port (such as 5022), you can append the port number to the address like the example below.

In [ ]:
from azureml.core.compute import DsvmCompute
from azureml.core.compute_target import ComputeTargetException

compute_target_name = 'mydsvm'

try:
    dsvm_compute = DsvmCompute(workspace = ws, name = compute_target_name)
    print('found existing:', dsvm_compute.name)
except ComputeTargetException:
    print('creating new.')
    dsvm_config = DsvmCompute.provisioning_configuration(vm_size = "Standard_D2_v2")
    dsvm_compute = DsvmCompute.create(ws, name = compute_target_name, provisioning_configuration = dsvm_config)
    dsvm_compute.wait_for_completion(show_output = True)

## Attach an existing Linux DSVM as a compute target


In [ ]:
'''
 from azureml.core.compute import RemoteCompute 
 # if you want to connect using SSH key instead of username/password you can provide parameters private_key_file and private_key_passphrase 
 dsvm_compute = RemoteCompute.attach(ws,name="attach-from-sdk6",username=<username>,address=<ipaddress>,ssh_port=22,password=<password>)
'''

## Configure & Run

### Configure a Docker run with new conda environment on the VM
You can execute in a Docker container in the VM. If you choose this route, you don't need to install anything on the VM yourself. Azure ML execution service will take care of it for you.

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies


# Load the "cpu-dsvm.runconfig" file (created by the above attach operation) in memory
run_config = RunConfiguration(framework = "python")

# Set compute target to the Linux DSVM
run_config.target = compute_target_name

# Use Docker in the remote VM
run_config.environment.docker.enabled = True

# Use CPU base image from DockerHub
run_config.environment.docker.base_image = azureml.core.runconfig.DEFAULT_CPU_IMAGE
print('Base Docker image is:', run_config.environment.docker.base_image)

# Ask system to provision a new one based on the conda_dependencies.yml file
run_config.environment.python.user_managed_dependencies = False

# Prepare the Docker and conda environment automatically when executingfor the first time.
run_config.prepare_environment = True

# specify CondaDependencies obj
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

### Submit the Experiment
Submit script to run in the Docker image in the remote VM. If you run this for the first time, the system will download the base image, layer in packages specified in the `conda_dependencies.yml` file on top of the base image, create a container and then execute the script in the container.

In [ ]:
from azureml.core import Run
from azureml.core import ScriptRunConfig

src = ScriptRunConfig(source_directory = '.', script = 'train.py', run_config = run_config)
run = exp.submit(src)

### View run history details

In [ ]:
run

In [ ]:
run.wait_for_completion(show_output = True)

### Find the best run

In [ ]:
# get all metris logged in the run
run.get_metrics()
metrics = run.get_metrics()

In [ ]:
import numpy as np
print('When alpha is {1:0.2f}, we have min MSE {0:0.2f}.'.format(
    min(metrics['mse']), 
    metrics['alpha'][np.argmin(metrics['mse'])]
))

## Clean up compute resource

In [ ]:
dsvm_compute.delete()